In [2]:
import os

In [3]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\stb_pfe_mlflow\\research'

In [4]:
os.chdir("../")
%pwd

'c:\\Users\\Lenovo\\Desktop\\stb_pfe_mlflow'

In [5]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [22]:
from dataclasses import dataclass
from pathlib import Path


from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_neighbors: int
    weights: str
    algorithm: str
    p: int
    leaf_size: int
    target_column: str


In [23]:
from stb_pfe_mlflow.constants import *
from stb_pfe_mlflow.utils.common import read_yaml, create_directories

In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.kneighborsclassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_neighbors = params.n_neighbors,
            weights = params.weights,
            algorithm = params.algorithm,
            p = params.p,
            leaf_size = params.leaf_size,
            target_column = schema.name
            
        )

        return model_trainer_config

In [25]:
import pandas as pd
import os
from stb_pfe_mlflow import logger
from sklearn.linear_model import ElasticNet
import joblib

In [30]:
import pandas as pd
import os
import joblib
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Load the data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Separate features and target
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        # Encode the target column
        label_encoder = LabelEncoder()
        train_y_encoded = label_encoder.fit_transform(train_y)
        test_y_encoded = label_encoder.transform(test_y)

        # Initialize and train KNN model
        knn = KNeighborsClassifier(
            n_neighbors=self.config.n_neighbors,
            weights=self.config.weights,
            algorithm=self.config.algorithm,
            p=self.config.p,
            leaf_size=self.config.leaf_size
        )
        knn.fit(train_x, train_y_encoded)

        # Save the model
        joblib.dump(knn, os.path.join(self.config.root_dir, self.config.model_name))

        # Optionally: Save the label encoder for future use
        joblib.dump(label_encoder, os.path.join(self.config.root_dir,self.config.model_name))

        print("Model training complete and saved.")

# Assuming you have defined `ModelTrainerConfig` with appropriate attributes


In [31]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-09-13 02:23:07,255: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-13 02:23:07,255: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-13 02:23:07,262: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-13 02:23:07,263: INFO: common: created directory at: artifacts]
[2024-09-13 02:23:07,265: INFO: common: created directory at: artifacts/model_trainer]
Model training complete and saved.
